In [85]:
###Q5 final
sc.stop()
from pyspark import SparkContext
sc = SparkContext(appName="exercise")
def max_temperature(a,b):
    if a>=b:
        return a
    else:
        return b
    
precipitation_file=sc.textFile("precipitation-readings-tiny.csv")
linesp = precipitation_file.map(lambda line:line.split(";"))

stations_Ostergotland=sc.textFile("stations-Ostergotland.csv")
liness = stations_Ostergotland.map(lambda line:line.split(";"))

# print(liness.take(10))

In [86]:
######## the list of all the stations
stationNum=liness .map(lambda x:(x[0],1))
stationNum=stationNum.reduceByKey((lambda v1,v2:v1+v2))
# print(*stationNum.take(10),sep="\n")

######## the daily-temperatures we need for each stations
year_precipitation = linesp.map(lambda x:(tuple([x[1][0:7],x[0]]),float(x[3])))
year_precipitation = year_precipitation.filter(lambda x: int(x[0][0][0:4])>=1996 and int(x[0][0][0:4])<=2016)

######## the month-temperatures we need for each stations
month_precipitationSum = year_precipitation.reduceByKey(lambda v1,v2:v1+v2)
month_precipitationSum = month_precipitationSum.map(lambda x:(x[0][1],tuple([x[0][0],x[1]])))
# print(*month_precipitationSum.take(10),sep="\n")

######### join by station-number
month_precipitationSum = month_precipitationSum.join(stationNum)
print(*month_precipitationSum.take(10),sep="\n")

month_precipitationSum = month_precipitationSum.map(lambda x:(x[1][0][0],(x[1][0][1],1)) )

######### reduce by date, get the summation of temperatures with #stations
month_precipitation = month_precipitationSum.reduceByKey(lambda v1,v2:(v1[0]+v2[0],v1[1]+v2[1]))
print(*month_precipitation.take(10),sep="\n")

('84260', (('1996-10', 2.0), 1))
('85630', (('1996-10', 4.0), 1))
('1996-10', (6.0, 2))


In [87]:
aveprec = month_precipitation.map( lambda x: (x[0][0:4],x[0][5:7],round(x[1][0]/x[1][1],2)) )
print(*aveprec.take(10),sep="\n")

# aveprec.saveAsTextFile("aveprec")

('1996', '10', 3.0)
